### Import the necessary libraries

In [1]:
from requests import get, post
import pandas as pd

### API Keys
Add your API key here. We will pass it to any API calls we make through an header.

In [2]:
API_KEY = "0PiRuJ5J5tLj6YDJDBalzM410fsFS18I"
HEADER = {"x-dune-api-key" : API_KEY}

### Simplifying URL generation
Let us create a function to generate different URLs for different API endpoints. You would see this in action in the coming section.

In [3]:
BASE_URL = "https://api.dune.com/api/v1/"

def make_api_url(module, action, ID):
    """
    We shall use this function to generate a URL to call the API.
    """
    
    url = BASE_URL + module + "/" + ID + "/" + action
    
    return url

### Wrapping API endpoints in functions

> The Dune API currently has four primary end points. We are going to wrap these up in neat functions. Which shall then make it very easy to use the API. As you would see in the coming sections.

In [4]:
def execute_query(query_id):
    """
    Takes in the query ID.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """
    
    url = make_api_url("query", "execute", query_id)
    response = post(url, headers=HEADER)
    execution_id = response.json()['execution_id']
    
    return execution_id


def get_query_status(execution_id):
    """
    Takes in an execution ID.
    Fetches the status of query execution using the API
    Returns the status response object
    """
    
    url = make_api_url("execution", "status", execution_id)
    response = get(url, headers=HEADER)
    
    return response


def get_query_results(execution_id):
    """
    Takes in an execution ID.
    Fetches the results returned from the query using the API
    Returns the results response object
    """
    
    url = make_api_url("execution", "results", execution_id)
    response = get(url, headers=HEADER)
    
    return response


def cancel_query_execution(execution_id):
    """
    Takes in an execution ID.
    Cancels the ongoing execution of the query.
    Returns the response object.
    """
    
    url = make_api_url("execution", "cancel", execution_id)
    response = get(url, headers=HEADER)
    
    return response

# Using the API

### Execute a Query
You can pass any query ID from Dune you would like to fetch data from.

In [5]:
execution_id = execute_query("1258228")

 > This function returned an execution ID. Which we shall now pass to rest of the APIs through our functions.

In [6]:
execution_id

'01GCV1RTAWZKANHADAJ28RHRNY'

### Get Query Status

In [7]:
response = get_query_status(execution_id)

 > In this response object, you might primarily want to check the `state` which in our case is `QUERY_STATE_COMPLETED`. Indicating the query has completed execution.

In [8]:
response.json()

{'execution_id': '01GCV1RTAWZKANHADAJ28RHRNY',
 'query_id': 1258228,
 'state': 'QUERY_STATE_COMPLETED',
 'submitted_at': '2022-09-13T09:09:35.195804Z',
 'expires_at': '2024-09-12T09:09:35.220704Z',
 'execution_started_at': '2022-09-13T09:09:35.20666Z',
 'execution_ended_at': '2022-09-13T09:09:35.220702Z',
 'result_metadata': {'column_names': ['block_time',
   'token_a_symbol',
   'token_b_symbol',
   'token_a_amount',
   'token_b_amount',
   'project',
   'version',
   'category',
   'trader_a',
   'trader_b',
   'token_a_amount_raw',
   'token_b_amount_raw',
   'usd_amount',
   'token_a_address',
   'token_b_address',
   'exchange_contract_address',
   'tx_hash',
   'tx_from',
   'tx_to',
   'trace_address',
   'evt_index',
   'trade_id'],
  'result_set_bytes': 5023,
  'total_row_count': 10,
  'datapoint_count': 220,
  'pending_time_millis': 10,
  'execution_time_millis': 14}}

### Get Query results
Finally we would like to load the results from the execution of our query.

In [9]:
response = get_query_results(execution_id)

Lets wrap this data up in a neat pandas dataframe

In [10]:
data = pd.DataFrame(response.json()['result']['rows'])

And here is your data from your Dune query. In a table. In Python. Hurray!

In [11]:
data

,block_time,category,evt_index,exchange_contract_address,project,token_a_address,token_a_amount,token_a_amount_raw,token_a_symbol,token_b_address,...,token_b_symbol,trace_address,trade_id,trader_a,trader_b,tx_from,tx_hash,tx_to,usd_amount,version
0,2020-05-07T23:13:34+00:00,DEX,115,\x05cde89ccfa0ada8c88d5a23caaa79ef129e7883,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2.648136e+01,26481360361120055000,WETH,\x2b591e99afe9f32eaa6214f7b7629768c40eeb39,...,HEX,[],1,\x58e7286bf13da5d33b2401e535c45834b69c9c96,None,\x58e7286bf13da5d33b2401e535c45834b69c9c96,\xee2052b32b52071e771c1516dede986b6ade8c98d98a...,\x05cde89ccfa0ada8c88d5a23caaa79ef129e7883,5602.131784,1
1,2021-10-28T09:04:10+00:00,DEX,229,\x8867f20c1c63baccec7617626254a060eeb0e61e,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3.392347e-01,339234724736126100,WETH,\x6c5ba91642f10282b576d91922ae6448c9d52f4e,...,PHA,[],1,\x819f3450da6f110ba6ea52195b3beafa246062de,None,\xce09539a047270e507fdd71fb4a759e8f0db6187,\x14ff4f4e58b2f603c62620ea5236143f02b08b5ce020...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,1395.116375,2
2,2021-10-28T09:04:10+00:00,DEX,232,\x819f3450da6f110ba6ea52195b3beafa246062de,Uniswap,\x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,7.314747e+02,731474689042306400000,MATIC,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,WETH,[],1,\xce09539a047270e507fdd71fb4a759e8f0db6187,None,\xce09539a047270e507fdd71fb4a759e8f0db6187,\x14ff4f4e58b2f603c62620ea5236143f02b08b5ce020...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,1404.431403,2
3,2021-10-28T15:17:04+00:00,DEX,253,\xb9ad5ee1d62e2441d2c0f37734c4ea56d3d156b1,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.159083e+00,1159083338921542000,WETH,\x4f2ab9d03ce5b8d0d3bca09259c78005d2775e08,...,None,[],1,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,None,\x18c7d0f1668a44b2bab66079e444fc078678591f,\x14ff598d6bd614bd7ca2660d2b9a76e29db674246305...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,4847.124252,2
4,2020-05-06T03:13:37+00:00,DEX,93,\x05cde89ccfa0ada8c88d5a23caaa79ef129e7883,Uniswap,\x2b591e99afe9f32eaa6214f7b7629768c40eeb39,1.250000e+05,12500000000000,HEX,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,WETH,[],1,\xfc96e234d4b31c63051e707105fcc4aba37807fa,None,\x2e15068e4027f124194dd0e003badf5adb8b94c4,\xee2108bc8c6fb0b241ecae70afd41840b284615d593c...,\xfc96e234d4b31c63051e707105fcc4aba37807fa,692.125000,1
5,2020-05-06T03:13:37+00:00,DEX,94,\xfc96e234d4b31c63051e707105fcc4aba37807fa,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3.403976e+00,3403975603973691400,WETH,\x8400d94a5cb0fa0d041a3788e395285d61c9ee5e,...,UBT,[],1,\x2e15068e4027f124194dd0e003badf5adb8b94c4,None,\x2e15068e4027f124194dd0e003badf5adb8b94c4,\xee2108bc8c6fb0b241ecae70afd41840b284615d593c...,\xfc96e234d4b31c63051e707105fcc4aba37807fa,694.138705,1
6,2021-10-27T02:35:40+00:00,DEX,124,\x163e3cd213327d9a121655341b56226d06be3636,Uniswap,\x2afe947da357679ae3e7d3b54bab4fa904d21c65,NaN,4350065314736938942464.0,None,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,WETH,[],1,\x9add984681ad1e0b0ed8f2d7e38c16fecdec29ee,None,\x9add984681ad1e0b0ed8f2d7e38c16fecdec29ee,\x14ff93e5ff33f0edf005f6be1ea3f5d8f6ab6f91a31f...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,3035.403000,2
7,2021-10-25T04:20:25+00:00,DEX,117,\x95332661b2e36a02dc05e6fb7c31193f0690c90f,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.195542e-01,119554189277620290,WETH,\xc1d9b5a0776d7c8b98b8a838e5a0dd1bc5fdd53c,...,ZONE,[],1,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,None,\x8b29f4a5c33344f8a429a4a39550decbe154642a,\x14ff9638732ff9416588eb7c67ae8e350d25ddbc8ab0...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,492.808346,2
8,2021-10-22T00:07:42+00:00,DEX,128,\xbc6d37be25b06471d9e094dbb800c1d006e5a9ed,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.447105e+00,1447104539321768700,WETH,\x4e0fca55a6c3a94720ded91153a27f60e26b9aa8,...,None,[],1,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,None,\x941420b447bfe645591fe85b373ed947a6d974fc,\x14ffab4c48bdb6cabc7e51169656e38444dd5d003bc7...,\x7a250d5630b4cf539

### Cancel Query Execution

 > Some queries can take a long time to execute (minutes), and you might want to interrupt this execution at times. This is how to do that.

In [12]:
response = cancel_query_execution(execution_id)

When you have a running query and call this function. You would get a response object returned to you confirming the cancellation of query execution.